In [1]:
# Simulation - Book all available suites for 10/12/2025, 13:00-16:00 (3 hours)
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Rate ID for JOURNEE (base weekday rate)
JOURNEE_RATE_ID = "c3c2109d-984a-4ad4-978e-b3850108b8ad"

# Age category for adults in JOURNEE service (IMPORTANT: Different from NUITEE!)
ADULT_AGE_CATEGORY_ID = "a78b7aca-fa0b-4199-8b4e-b3850108b8a5"

# Booking details
# 10/12/2025 13:00 - 16:00 (3 hours) in UTC
START_UTC = "2025-12-10T13:00:00Z"
END_UTC = "2025-12-10T16:00:00Z"

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

# Step 1: Fetch all suites for JOURNEE service
print("="*80)
print("Step 1: Fetching all suites for JOURNEE service")
print("="*80)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

# Filter to only show suites and rooms (not buildings/floors/private spaces)
all_suites = [cat for cat in suites_result["ResourceCategories"]
              if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]

print(f"✅ Found {len(all_suites)} suites/rooms for JOURNEE service:")
for suite in all_suites:
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    suite_type = suite.get("Type")
    print(f"   - {suite_name} ({suite_type}): {suite['Id']}")

SUITE_IDS = [suite["Id"] for suite in all_suites]
print()

# Step 2: Create customer with unique timestamp
print("="*80)
print("Step 2: Creating customer with unique timestamp")
print("="*80)

# Generate unique timestamp
timestamp = int(time.time())
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

customer_payload = {
    "Client": "Intense Experience Simulation",
    "FirstName": f"Simulation",
    "LastName": f"User_{timestamp_str}",
    "Email": f"simulation_{timestamp}@intense-experience.test",
    "Phone": f"+32{timestamp % 1000000000}",  # Use last 9 digits of timestamp for phone
    "OverwriteExisting": True
}

print(f"Creating customer:")
print(f"  Name: {customer_payload['FirstName']} {customer_payload['LastName']}")
print(f"  Email: {customer_payload['Email']}")
print(f"  Phone: {customer_payload['Phone']}")
print()

customer_result = make_mews_request("customers/add", customer_payload)

if not customer_result or 'Id' not in customer_result:
    print("❌ Failed to create customer")
    print(json.dumps(customer_result, indent=2))
else:
    customer_id = customer_result['Id']
    print(f"✅ Customer created successfully")
    print(f"   Customer ID: {customer_id}")
    print(f"   Name: {customer_result.get('FirstName')} {customer_result.get('LastName')}")
    print(f"   Email: {customer_result.get('Email')}")
    print()
    
    # Step 3: Create reservations for all suites
    print("="*80)
    print("Step 3: Creating reservations for all suites")
    print(f"Date: 10/12/2025")
    print(f"Time: 13:00 - 16:00 (3 hours)")
    print(f"Service: JOURNEE")
    print("="*80)
    print()
    
    reservations_created = []
    reservations_failed = []
    
    # Create a lookup dictionary for suite names
    suite_names = {suite["Id"]: suite.get("Names", {}).get("fr-FR", "Unknown") for suite in all_suites}
    
    for suite_id in SUITE_IDS:
        suite_name = suite_names.get(suite_id, suite_id)
        print(f"Creating reservation for {suite_name}...")
        
        # Create unique identifier for this reservation
        suite_code = suite_name.replace(' ', '-').replace('Suite', 'S').replace('Chambre', 'C')
        
        reservation_data = {
            "Identifier": f"SIM-{timestamp_str}-{suite_code}",
            "State": "Confirmed",
            "StartUtc": START_UTC,
            "EndUtc": END_UTC,
            "CustomerId": customer_id,
            "BookerId": customer_id,
            "PersonCounts": [
                {
                    "AgeCategoryId": ADULT_AGE_CATEGORY_ID,
                    "Count": 2
                }
            ],
            "RequestedCategoryId": suite_id,
            "RateId": JOURNEE_RATE_ID
        }
        
        reservation_payload = {
            "Client": "Intense Experience Simulation",
            "ServiceId": JOURNEE_SERVICE_ID,
            "Reservations": [reservation_data]
        }
        
        reservation_result = make_mews_request("reservations/add", reservation_payload)
        
        if reservation_result and "Reservations" in reservation_result and reservation_result["Reservations"]:
            reservation = reservation_result["Reservations"][0]
            reservations_created.append({
                "suite": suite_name,
                "reservation_id": reservation["Reservation"]["Id"],
                "number": reservation["Reservation"]["Number"]
            })
            print(f"   ✅ {suite_name}: Reservation #{reservation['Reservation']['Number']} created")
            print(f"      ID: {reservation['Reservation']['Id']}")
        else:
            reservations_failed.append({
                "suite": suite_name,
                "error": reservation_result
            })
            print(f"   ❌ {suite_name}: Failed to create reservation")
            if reservation_result:
                print(f"      Error: {json.dumps(reservation_result, indent=6)}")
        
        print()
    
    # Step 4: Summary
    print("="*80)
    print("SUMMARY")
    print("="*80)
    print(f"Customer: {customer_payload['FirstName']} {customer_payload['LastName']}")
    print(f"  ID: {customer_id}")
    print(f"  Email: {customer_payload['Email']}")
    print(f"  Phone: {customer_payload['Phone']}")
    print()
    print(f"Date: 10/12/2025 (Wednesday)")
    print(f"Time: 13:00 - 16:00 (3 hours)")
    print(f"Service: JOURNEE")
    print()
    print(f"✅ Reservations created: {len(reservations_created)}")
    for res in reservations_created:
        print(f"   - {res['suite']}: Reservation #{res['number']} (ID: {res['reservation_id']})")
    
    if reservations_failed:
        print()
        print(f"❌ Reservations failed: {len(reservations_failed)}")
        for res in reservations_failed:
            print(f"   - {res['suite']}")
    
    print("="*80)
    print(f"\n🎯 Simulation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔑 Timestamp identifier: {timestamp_str}")


Step 1: Fetching all suites for JOURNEE service
✅ Found 7 suites/rooms for JOURNEE service:
   - Suite GAIA (Suite): f723bd5a-04fe-479c-bab4-b3850108c66f
   - Chambre EUPHORYA (Room): f5539e51-9db0-4082-b87e-b3850108c66f
   - Suite EXTASE (Suite): 68b87fe5-7b78-4067-b5e7-b3850108c66f
   - Chambre IGNIS (Room): 78a614b1-199d-4608-ab89-b3850108c66f
   - Suite INTENSE (Suite): e4706d3a-2a06-4cb7-a449-b3850108c66f
   - Chambre KAIROS (Room): 67ece5a2-65e2-43c5-9079-b3850108c66f
   - Chambre AETHER (Room): 1113bcbe-ad5f-49c7-8dc0-b3850108c66f

Step 2: Creating customer with unique timestamp
Creating customer:
  Name: Simulation User_20251128_230840
  Email: simulation_1764367720@intense-experience.test
  Phone: +32764367720

✅ Customer created successfully
   Customer ID: 5675d6f8-ce15-4743-89cd-b3a3016ceed1
   Name: Simulation User_20251128_230840
   Email: simulation_1764367720@intense-experience.test

Step 3: Creating reservations for all suites
Date: 10/12/2025
Time: 13:00 - 16:00 (3 ho

In [3]:

# Simulation - Book all available suites for 18/12/2025, 10:00-16:00 (6 hours)
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Rate ID for JOURNEE (base weekday rate)
JOURNEE_RATE_ID = "c3c2109d-984a-4ad4-978e-b3850108b8ad"

# Age category for adults in JOURNEE service (IMPORTANT: Different from NUITEE!)
ADULT_AGE_CATEGORY_ID = "a78b7aca-fa0b-4199-8b4e-b3850108b8a5"

# Booking details
# 18/12/2025 10:00 - 16:00 (6 hours) in UTC
START_UTC = "2025-12-18T10:00:00Z"
END_UTC = "2025-12-18T16:00:00Z"

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

# Step 1: Fetch all suites for JOURNEE service
print("="*80)
print("Step 1: Fetching all suites for JOURNEE service")
print("="*80)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

# Filter to only show suites and rooms (not buildings/floors/private spaces)
all_suites = [cat for cat in suites_result["ResourceCategories"]
              if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]

print(f"✅ Found {len(all_suites)} suites/rooms for JOURNEE service:")
for suite in all_suites:
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    suite_type = suite.get("Type")
    print(f"   - {suite_name} ({suite_type}): {suite['Id']}")

SUITE_IDS = [suite["Id"] for suite in all_suites]
print()

# Step 2: Create customer with unique timestamp
print("="*80)
print("Step 2: Creating customer with unique timestamp")
print("="*80)

# Generate unique timestamp
timestamp = int(time.time())
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

customer_payload = {
    "Client": "Intense Experience Simulation",
    "FirstName": f"Simulation",
    "LastName": f"User_{timestamp_str}",
    "Email": f"simulation_{timestamp}@intense-experience.test",
    "Phone": f"+32{timestamp % 1000000000}",  # Use last 9 digits of timestamp for phone
    "OverwriteExisting": True
}

print(f"Creating customer:")
print(f"  Name: {customer_payload['FirstName']} {customer_payload['LastName']}")
print(f"  Email: {customer_payload['Email']}")
print(f"  Phone: {customer_payload['Phone']}")
print()

customer_result = make_mews_request("customers/add", customer_payload)

if not customer_result or 'Id' not in customer_result:
    print("❌ Failed to create customer")
    print(json.dumps(customer_result, indent=2))
else:
    customer_id = customer_result['Id']
    print(f"✅ Customer created successfully")
    print(f"   Customer ID: {customer_id}")
    print(f"   Name: {customer_result.get('FirstName')} {customer_result.get('LastName')}")
    print(f"   Email: {customer_result.get('Email')}")
    print()
    
    # Step 3: Create reservations for all suites
    print("="*80)
    print("Step 3: Creating reservations for all suites")
    print(f"Date: 18/12/2025")
    print(f"Time: 10:00 - 16:00 (6 hours)")
    print(f"Service: JOURNEE")
    print("="*80)
    print()
    
    reservations_created = []
    reservations_failed = []
    
    # Create a lookup dictionary for suite names
    suite_names = {suite["Id"]: suite.get("Names", {}).get("fr-FR", "Unknown") for suite in all_suites}
    
    for suite_id in SUITE_IDS:
        suite_name = suite_names.get(suite_id, suite_id)
        print(f"Creating reservation for {suite_name}...")
        
        # Create unique identifier for this reservation
        suite_code = suite_name.replace(' ', '-').replace('Suite', 'S').replace('Chambre', 'C')
        
        reservation_data = {
            "Identifier": f"SIM-{timestamp_str}-{suite_code}",
            "State": "Confirmed",
            "StartUtc": START_UTC,
            "EndUtc": END_UTC,
            "CustomerId": customer_id,
            "BookerId": customer_id,
            "PersonCounts": [
                {
                    "AgeCategoryId": ADULT_AGE_CATEGORY_ID,
                    "Count": 2
                }
            ],
            "RequestedCategoryId": suite_id,
            "RateId": JOURNEE_RATE_ID
        }
        
        reservation_payload = {
            "Client": "Intense Experience Simulation",
            "ServiceId": JOURNEE_SERVICE_ID,
            "Reservations": [reservation_data]
        }
        
        reservation_result = make_mews_request("reservations/add", reservation_payload)
        
        if reservation_result and "Reservations" in reservation_result and reservation_result["Reservations"]:
            reservation = reservation_result["Reservations"][0]
            reservations_created.append({
                "suite": suite_name,
                "reservation_id": reservation["Reservation"]["Id"],
                "number": reservation["Reservation"]["Number"]
            })
            print(f"   ✅ {suite_name}: Reservation #{reservation['Reservation']['Number']} created")
            print(f"      ID: {reservation['Reservation']['Id']}")
        else:
            reservations_failed.append({
                "suite": suite_name,
                "error": reservation_result
            })
            print(f"   ❌ {suite_name}: Failed to create reservation")
            if reservation_result:
                print(f"      Error: {json.dumps(reservation_result, indent=6)}")
        
        print()
    
    # Step 4: Summary
    print("="*80)
    print("SUMMARY")
    print("="*80)
    print(f"Customer: {customer_payload['FirstName']} {customer_payload['LastName']}")
    print(f"  ID: {customer_id}")
    print(f"  Email: {customer_payload['Email']}")
    print(f"  Phone: {customer_payload['Phone']}")
    print()
    print(f"Date: 18/12/2025 (Thursday)")
    print(f"Time: 10:00 - 16:00 (6 hours)")
    print(f"Service: JOURNEE")
    print()
    print(f"✅ Reservations created: {len(reservations_created)}")
    for res in reservations_created:
        print(f"   - {res['suite']}: Reservation #{res['number']} (ID: {res['reservation_id']})")
    
    if reservations_failed:
        print()
        print(f"❌ Reservations failed: {len(reservations_failed)}")
        for res in reservations_failed:
            print(f"   - {res['suite']}")
    
    print("="*80)
    print(f"\n🎯 Simulation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔑 Timestamp identifier: {timestamp_str}")


Step 1: Fetching all suites for JOURNEE service
✅ Found 7 suites/rooms for JOURNEE service:
   - Suite GAIA (Suite): f723bd5a-04fe-479c-bab4-b3850108c66f
   - Chambre EUPHORYA (Room): f5539e51-9db0-4082-b87e-b3850108c66f
   - Suite EXTASE (Suite): 68b87fe5-7b78-4067-b5e7-b3850108c66f
   - Chambre IGNIS (Room): 78a614b1-199d-4608-ab89-b3850108c66f
   - Suite INTENSE (Suite): e4706d3a-2a06-4cb7-a449-b3850108c66f
   - Chambre KAIROS (Room): 67ece5a2-65e2-43c5-9079-b3850108c66f
   - Chambre AETHER (Room): 1113bcbe-ad5f-49c7-8dc0-b3850108c66f

Step 2: Creating customer with unique timestamp
Creating customer:
  Name: Simulation User_20251122_114837
  Email: simulation_1763808517@intense-experience.test
  Phone: +32763808517

✅ Customer created successfully
   Customer ID: d120dccd-e2a2-467c-8ebc-b39d00b22744
   Name: Simulation User_20251122_114837
   Email: simulation_1763808517@intense-experience.test

Step 3: Creating reservations for all suites
Date: 18/12/2025
Time: 10:00 - 16:00 (6 ho

In [ ]:
# Check availability for December 16th and 17th, 2025 (JOURNEE service)
import requests
import json
from datetime import datetime, timedelta, timezone
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Booking constraints
DAY_MIN_HOURS = 3
DAY_MAX_HOURS = 6
CLEANING_BUFFER_HOURS = 1
ARRIVAL_TIMES = ['13:00', '14:00', '15:00', '16:00', '17:00']
DEPARTURE_TIMES = ['14:00', '15:00', '16:00', '17:00', '18:00']

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

print("="*100)
print("AVAILABILITY CHECK FOR DECEMBER 16-17, 2025 (JOURNEE SERVICE)")
print("="*100)
print()

# Step 1: Fetch all suites for JOURNEE service
print("Step 1: Fetching all suites for JOURNEE service")
print("-"*100)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

all_suites = [cat for cat in suites_result["ResourceCategories"]
              if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]
suite_ids = [suite["Id"] for suite in all_suites]

print(f"✅ Found {len(all_suites)} active suites/rooms:")
for suite in all_suites:
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    suite_type = suite.get("Type")
    suite_id_short = suite['Id'][:8]
    print(f"   - {suite_name} ({suite_type}): {suite_id_short}...")

print()

# Step 2: Check availability for December 16-17
dates_to_check = [
    datetime(2025, 12, 16, 0, 0, 0, tzinfo=timezone.utc),
    datetime(2025, 12, 17, 0, 0, 0, tzinfo=timezone.utc)
]

print("Step 2: Fetching reservations for December 16-17, 2025")
print("-"*100)

# Fetch reservations for the date range (with buffer)
start_date = dates_to_check[0]
end_date = dates_to_check[-1] + timedelta(days=1)
buffered_end = end_date + timedelta(hours=CLEANING_BUFFER_HOURS)

print(f"Query range: {start_date.strftime('%Y-%m-%d')} to {buffered_end.strftime('%Y-%m-%d %H:%M')} (with buffer)")
print()

reservations_payload = {
    "Client": "Intense Experience Availability Check",
    "StartUtc": start_date.isoformat(),
    "EndUtc": buffered_end.isoformat()
}

reservations_result = make_mews_request("reservations/getAll", reservations_payload)

if not reservations_result:
    print("❌ Failed to fetch reservations")
    exit()

reservations = reservations_result.get("Reservations", [])
print(f"✅ Found {len(reservations)} total reservations in date range")
print()

# Filter reservations by suite
suite_reservations = {}
for suite_id in suite_ids:
    suite_reservations[suite_id] = []
    for res in reservations:
        if res.get('RequestedCategoryId') == suite_id:
            suite_reservations[suite_id].append(res)

print("Reservations by suite:")
for suite in all_suites:
    suite_id = suite['Id']
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    res_count = len(suite_reservations[suite_id])
    print(f"   - {suite_name}: {res_count} reservation(s)")
    for res in suite_reservations[suite_id]:
        res_start = datetime.fromisoformat(res.get('StartUtc', '').replace('Z', '+00:00'))
        res_end = datetime.fromisoformat(res.get('EndUtc', '').replace('Z', '+00:00'))
        res_id_short = res.get('Id', 'unknown')[:8]
        print(f"      • {res_id_short}: {res_start.strftime('%Y-%m-%d %H:%M')} → {res_end.strftime('%H:%M')}")

print()
print("="*100)

# Step 3: Check availability for each date
for check_date in dates_to_check:
    date_str = check_date.strftime('%Y-%m-%d')
    day_name = check_date.strftime('%A')
    
    print()
    print(f"📅 AVAILABILITY ANALYSIS FOR {date_str} ({day_name})")
    print("="*100)
    
    suite_availability = {}
    
    # Check each suite
    for suite in all_suites:
        suite_id = suite['Id']
        suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
        available_slots = []
        blocked_slots = []
        
        # Check all time slot combinations
        for arrival_time in ARRIVAL_TIMES:
            for departure_time in DEPARTURE_TIMES:
                # Calculate duration
                arr_hour = int(arrival_time.split(':')[0])
                dep_hour = int(departure_time.split(':')[0])
                duration = dep_hour - arr_hour
                
                # Check if duration is within limits
                if duration < DAY_MIN_HOURS or duration > DAY_MAX_HOURS:
                    continue
                
                # Create datetime objects for this slot
                slot_start = datetime.combine(check_date.date(), datetime.strptime(arrival_time, '%H:%M').time()).replace(tzinfo=timezone.utc)
                slot_end = datetime.combine(check_date.date(), datetime.strptime(departure_time, '%H:%M').time()).replace(tzinfo=timezone.utc)
                slot_end_buffered = slot_end + timedelta(hours=CLEANING_BUFFER_HOURS)
                
                # Check if this slot conflicts with any reservation
                is_available = True
                conflicting_reservation = None
                
                for reservation in suite_reservations[suite_id]:
                    res_start = datetime.fromisoformat(reservation.get('StartUtc', '').replace('Z', '+00:00'))
                    res_end = datetime.fromisoformat(reservation.get('EndUtc', '').replace('Z', '+00:00'))
                    
                    # Check for overlap (including buffer)
                    if not (slot_end_buffered <= res_start or slot_start >= res_end):
                        is_available = False
                        conflicting_reservation = reservation
                        break
                
                if is_available:
                    available_slots.append({
                        'arrival': arrival_time,
                        'departure': departure_time,
                        'duration': duration
                    })
                else:
                    blocked_slots.append({
                        'arrival': arrival_time,
                        'departure': departure_time,
                        'duration': duration,
                        'conflict': conflicting_reservation
                    })
        
        suite_availability[suite_id] = {
            'name': suite_name,
            'available': available_slots,
            'blocked': blocked_slots
        }
    
    # Summary for this date
    total_available_slots = sum(len(data['available']) for data in suite_availability.values())
    has_availability = total_available_slots > 0
    
    print(f"\n🎯 SUMMARY:")
    print(f"   Total suites checked: {len(all_suites)}")
    print(f"   Total available time slots: {total_available_slots}")
    print(f"   Status: {'✅ AVAILABLE' if has_availability else '❌ FULLY BOOKED'}")
    print()
    
    # Detailed breakdown by suite
    print(f"📊 SUITE-BY-SUITE BREAKDOWN:")
    print()
    
    for suite in all_suites:
        suite_id = suite['Id']
        data = suite_availability[suite_id]
        suite_name = data['name']
        num_available = len(data['available'])
        num_blocked = len(data['blocked'])
        
        if num_available > 0:
            print(f"✅ {suite_name}: {num_available} available slot(s)")
            print(f"   Available time slots:")
            for slot in data['available']:
                print(f"      • {slot['arrival']} → {slot['departure']} ({slot['duration']}h)")
        else:
            print(f"❌ {suite_name}: NO available slots (fully booked)")
        
        if num_blocked > 0:
            print(f"   Blocked time slots: {num_blocked}")
            # Show first 3 blocked slots with details
            for i, slot in enumerate(data['blocked'][:3]):
                conflict = slot['conflict']
                res_id_short = conflict.get('Id', 'unknown')[:8]
                res_start = datetime.fromisoformat(conflict.get('StartUtc', '').replace('Z', '+00:00'))
                res_end = datetime.fromisoformat(conflict.get('EndUtc', '').replace('Z', '+00:00'))
                print(f"      ✗ {slot['arrival']} → {slot['departure']} blocked by reservation {res_id_short}")
                print(f"         Conflicting reservation: {res_start.strftime('%Y-%m-%d %H:%M')} → {res_end.strftime('%H:%M')}")
            
            if num_blocked > 3:
                print(f"      ... and {num_blocked - 3} more blocked slots")
        
        print()
    
    print("-"*100)

print()
print("="*100)
print("✅ AVAILABILITY CHECK COMPLETE")
print("="*100)


AVAILABILITY CHECK FOR DECEMBER 16-17, 2025 (JOURNEE SERVICE)

Step 1: Fetching all suites for JOURNEE service
----------------------------------------------------------------------------------------------------
✅ Found 7 active suites/rooms:
   - Suite GAIA (Suite): f723bd5a...
   - Chambre EUPHORYA (Room): f5539e51...
   - Suite EXTASE (Suite): 68b87fe5...
   - Chambre IGNIS (Room): 78a614b1...
   - Suite INTENSE (Suite): e4706d3a...
   - Chambre KAIROS (Room): 67ece5a2...
   - Chambre AETHER (Room): 1113bcbe...

Step 2: Fetching reservations for December 16-17, 2025
----------------------------------------------------------------------------------------------------
Query range: 2025-12-16 to 2025-12-18 01:00 (with buffer)

✅ Found 8 total reservations in date range

Reservations by suite:
   - Suite GAIA: 1 reservation(s)
      • e309ae21: 2025-12-17 13:00 → 16:00
   - Chambre EUPHORYA: 1 reservation(s)
      • 3796c3a7: 2025-12-17 13:00 → 16:00
   - Suite EXTASE: 1 reservation(s)
 

In [1]:
# Check availability for December 5th, 2025 (JOURNEE service) - Suite e4706d3a only
import requests
import json
from datetime import datetime, timedelta, timezone
import os
from dotenv import load_dotenv
import pytz

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Target suite ID
TARGET_SUITE_ID = "e4706d3a-2a06-4cb7-a449-b3850108c66f"

# Booking constraints
DAY_MIN_HOURS = 3
DAY_MAX_HOURS = 6
CLEANING_BUFFER_HOURS = 1
ARRIVAL_TIMES = ['09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00']
DEPARTURE_TIMES = ['10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00']

# Brussels timezone
BRUSSELS_TZ = pytz.timezone('Europe/Brussels')

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

print("="*100)
print("AVAILABILITY CHECK FOR DECEMBER 5, 2025 (JOURNEE SERVICE) - SUITE e4706d3a ONLY")
print("="*100)
print()

# Step 1: Fetch suite information
print("Step 1: Fetching suite information")
print("-"*100)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

# Find the target suite
target_suite = None
for cat in suites_result["ResourceCategories"]:
    if cat.get("Id") == TARGET_SUITE_ID and cat.get("IsActive"):
        target_suite = cat
        break

if not target_suite:
    print(f"❌ Suite {TARGET_SUITE_ID} not found or not active")
    exit()

suite_name = target_suite.get("Names", {}).get("fr-FR", "Unknown")
suite_type = target_suite.get("Type")
print(f"✅ Found target suite: {suite_name} ({suite_type})")
print(f"   Suite ID: {TARGET_SUITE_ID}")
print()

# Step 2: Check availability for December 5
# Create date in Brussels timezone
check_date = BRUSSELS_TZ.localize(datetime(2025, 12, 5, 0, 0, 0))
date_str = check_date.strftime('%Y-%m-%d')
day_name = check_date.strftime('%A')

print("Step 2: Fetching reservations for December 5, 2025")
print("-"*100)

# Fetch reservations for the date (with buffer)
start_date = check_date
end_date = check_date + timedelta(days=1)
buffered_end = end_date + timedelta(hours=CLEANING_BUFFER_HOURS)

# Convert to UTC for API request
start_date_utc = start_date.astimezone(timezone.utc)
buffered_end_utc = buffered_end.astimezone(timezone.utc)

print(f"Query range (Brussels time): {start_date.strftime('%Y-%m-%d %H:%M %Z')} to {buffered_end.strftime('%Y-%m-%d %H:%M %Z')} (with buffer)")
print(f"Query range (UTC): {start_date_utc.strftime('%Y-%m-%d %H:%M %Z')} to {buffered_end_utc.strftime('%Y-%m-%d %H:%M %Z')}")
print()

reservations_payload = {
    "Client": "Intense Experience Availability Check",
    "StartUtc": start_date_utc.isoformat(),
    "EndUtc": buffered_end_utc.isoformat()
}

reservations_result = make_mews_request("reservations/getAll", reservations_payload)

if not reservations_result:
    print("❌ Failed to fetch reservations")
    exit()

reservations = reservations_result.get("Reservations", [])
print(f"✅ Found {len(reservations)} total reservations in date range")
print()

# Filter reservations for target suite
suite_reservations = [res for res in reservations if res.get('RequestedCategoryId') == TARGET_SUITE_ID]

print(f"Reservations for {suite_name}:")
if len(suite_reservations) == 0:
    print("   No reservations found")
else:
    for res in suite_reservations:
        res_start_utc = datetime.fromisoformat(res.get('StartUtc', '').replace('Z', '+00:00'))
        res_end_utc = datetime.fromisoformat(res.get('EndUtc', '').replace('Z', '+00:00'))
        # Convert to Brussels time for display
        res_start = res_start_utc.astimezone(BRUSSELS_TZ)
        res_end = res_end_utc.astimezone(BRUSSELS_TZ)
        res_id_short = res.get('Id', 'unknown')[:8]
        print(f"   • {res_id_short}: {res_start.strftime('%Y-%m-%d %H:%M %Z')} → {res_end.strftime('%H:%M %Z')}")

print()
print("="*100)

# Step 3: Check availability for December 5
print()
print(f"📅 AVAILABILITY ANALYSIS FOR {date_str} ({day_name}) - Brussels Time")
print("="*100)

available_slots = []
blocked_slots = []

# Check all time slot combinations
for arrival_time in ARRIVAL_TIMES:
    for departure_time in DEPARTURE_TIMES:
        # Calculate duration
        arr_hour = int(arrival_time.split(':')[0])
        dep_hour = int(departure_time.split(':')[0])
        duration = dep_hour - arr_hour
        
        # Check if duration is within limits
        if duration < DAY_MIN_HOURS or duration > DAY_MAX_HOURS:
            continue
        
        # Create datetime objects for this slot in Brussels timezone
        slot_start = BRUSSELS_TZ.localize(datetime.combine(check_date.date(), datetime.strptime(arrival_time, '%H:%M').time()))
        slot_end = BRUSSELS_TZ.localize(datetime.combine(check_date.date(), datetime.strptime(departure_time, '%H:%M').time()))
        slot_end_buffered = slot_end + timedelta(hours=CLEANING_BUFFER_HOURS)
        
        # Check if this slot conflicts with any reservation
        is_available = True
        conflicting_reservation = None
        
        for reservation in suite_reservations:
            res_start_utc = datetime.fromisoformat(reservation.get('StartUtc', '').replace('Z', '+00:00'))
            res_end_utc = datetime.fromisoformat(reservation.get('EndUtc', '').replace('Z', '+00:00'))
            # Convert to Brussels time for comparison
            res_start = res_start_utc.astimezone(BRUSSELS_TZ)
            res_end = res_end_utc.astimezone(BRUSSELS_TZ)
            
            # Check for overlap (including buffer)
            if not (slot_end_buffered <= res_start or slot_start >= res_end):
                is_available = False
                conflicting_reservation = reservation
                break
        
        if is_available:
            available_slots.append({
                'arrival': arrival_time,
                'departure': departure_time,
                'duration': duration
            })
        else:
            blocked_slots.append({
                'arrival': arrival_time,
                'departure': departure_time,
                'duration': duration,
                'conflict': conflicting_reservation
            })

# Summary
total_available_slots = len(available_slots)
has_availability = total_available_slots > 0

print(f"\n🎯 SUMMARY:")
print(f"   Suite: {suite_name}")
print(f"   Total available time slots: {total_available_slots}")
print(f"   Status: {'✅ AVAILABLE' if has_availability else '❌ FULLY BOOKED'}")
print()

# Detailed breakdown
print(f"📊 DETAILED BREAKDOWN:")
print()

if total_available_slots > 0:
    print(f"✅ {suite_name}: {total_available_slots} available slot(s)")
    print(f"   Available time slots:")
    for slot in available_slots:
        print(f"      • {slot['arrival']} → {slot['departure']} ({slot['duration']}h)")
else:
    print(f"❌ {suite_name}: NO available slots (fully booked)")

if len(blocked_slots) > 0:
    print(f"\n   Blocked time slots: {len(blocked_slots)}")
    # Show first 3 blocked slots with details
    for i, slot in enumerate(blocked_slots[:3]):
        conflict = slot['conflict']
        res_id_short = conflict.get('Id', 'unknown')[:8]
        res_start_utc = datetime.fromisoformat(conflict.get('StartUtc', '').replace('Z', '+00:00'))
        res_end_utc = datetime.fromisoformat(conflict.get('EndUtc', '').replace('Z', '+00:00'))
        # Convert to Brussels time for display
        res_start = res_start_utc.astimezone(BRUSSELS_TZ)
        res_end = res_end_utc.astimezone(BRUSSELS_TZ)
        print(f"      ✗ {slot['arrival']} → {slot['departure']} blocked by reservation {res_id_short}")
        print(f"         Conflicting reservation: {res_start.strftime('%Y-%m-%d %H:%M %Z')} → {res_end.strftime('%H:%M %Z')}")
    
    if len(blocked_slots) > 3:
        print(f"      ... and {len(blocked_slots) - 3} more blocked slots")

print()
print("-"*100)

print()
print("="*100)
print("✅ AVAILABILITY CHECK COMPLETE")
print("="*100)


AVAILABILITY CHECK FOR DECEMBER 5, 2025 (JOURNEE SERVICE) - SUITE e4706d3a ONLY

Step 1: Fetching suite information
----------------------------------------------------------------------------------------------------
✅ Found target suite: Suite INTENSE (Suite)
   Suite ID: e4706d3a-2a06-4cb7-a449-b3850108c66f

Step 2: Fetching reservations for December 5, 2025
----------------------------------------------------------------------------------------------------
Query range (Brussels time): 2025-12-05 00:00 CET to 2025-12-06 01:00 CET (with buffer)
Query range (UTC): 2025-12-04 23:00 UTC to 2025-12-06 00:00 UTC

✅ Found 1 total reservations in date range

Reservations for Suite INTENSE:
   • 1beff2f8: 2025-12-05 09:00 CET → 12:00 CET


📅 AVAILABILITY ANALYSIS FOR 2025-12-05 (Friday) - Brussels Time

🎯 SUMMARY:
   Suite: Suite INTENSE
   Total available time slots: 10
   Status: ✅ AVAILABLE

📊 DETAILED BREAKDOWN:

✅ Suite INTENSE: 10 available slot(s)
   Available time slots:
      • 12:00

In [4]:
import requests
from datetime import datetime, timedelta
import pytz
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Mews API configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken')
ACCESS_TOKEN = os.getenv('AccessToken')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service IDs
NIGHT_SERVICE_ID = "7ba0b732-93cc-477a-861d-b3850108b730"  # NUITEE

# Gaia Suite ID for night service
GAIA_SUITE_ID = "3872869e-6278-4c64-aea8-b3850108c66f"

# Time configuration
CLEANING_BUFFER_HOURS = 1
NIGHT_CHECK_IN_HOUR = 19  # 19:00 (7:00 PM)
NIGHT_CHECK_OUT_HOUR = 10  # 10:00 (10:00 AM)

# Timezone
BRUSSELS_TZ = pytz.timezone('Europe/Brussels')

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Set specific dates to check (December 15th and 16th, 2024)
dates = [
    '2025-12-15',
    '2025-12-16'
]

print("=" * 80)
print(f"BULK NIGHT AVAILABILITY CHECK FOR GAIA SUITE")
print("=" * 80)
print(f"Suite ID: {GAIA_SUITE_ID}")
print(f"Service: NUITEE ({NIGHT_SERVICE_ID})")
print(f"Dates to check: {len(dates)}")
print(f"Date range: {dates[0]} to {dates[-1]}")
print("=" * 80)
print()

# Prepare the data for bulk availability check
data = {
    'service_id': NIGHT_SERVICE_ID,
    'dates': dates,
    'booking_type': 'night',
    'suite_id': GAIA_SUITE_ID
}

# Import the function (assuming it's in the same directory)
from bulk_availability import check_bulk_availability_nuitee

# Call the function
result = check_bulk_availability_nuitee(make_mews_request, data)

# Print the results
print("\n" + "=" * 80)
print("RESULTS")
print("=" * 80)
print(f"\nStatus: {result.get('status')}")
print(f"\nTotal dates processed: {len(result.get('availability', {}))}")

if 'availability' in result:
    print("\n" + "-" * 80)
    print("DETAILED AVAILABILITY BY DATE:")
    print("-" * 80)
    
    for date_str in sorted(result['availability'].keys()):
        date_info = result['availability'][date_str]
        
        print(f"\n📅 Date: {date_str}")
        print(f"   Overall Available: {date_info.get('available')}")
        print(f"   Morning Available (00:00-10:00): {date_info.get('available_morning')}")
        print(f"   Night Available (19:00-10:00+1): {date_info.get('available_night')}")
        print(f"   Total Suites: {date_info.get('total_suites')}")
        print(f"   Booked Suites: {date_info.get('booked_suites')}")
        print(f"   Available Suites: {date_info.get('available_suites')}")
        
        if date_info.get('booked_suite_ids'):
            print(f"   Booked Suite IDs: {date_info.get('booked_suite_ids')}")

print("\n" + "=" * 80)
print("TIME SLOTS EXPLANATION:")
print("=" * 80)
print(f"Morning Slot: 00:00 - {NIGHT_CHECK_OUT_HOUR:02d}:00 (checkout period)")
print(f"Night Slot: {NIGHT_CHECK_IN_HOUR:02d}:00 - {NIGHT_CHECK_OUT_HOUR:02d}:00 next day (check-in period)")
print(f"Cleaning Buffer: {CLEANING_BUFFER_HOURS} hour(s)")
print("=" * 80)

# Print full JSON for detailed inspection
print("\n" + "=" * 80)
print("FULL JSON RESPONSE:")
print("=" * 80)
print(json.dumps(result, indent=2))

# Now fetch ALL reservations to see what's actually blocking December 15th night
print("\n" + "=" * 80)
print("DETAILED RESERVATION ANALYSIS FOR DECEMBER 15TH")
print("=" * 80)

# Query reservations around December 15th with buffer
date_obj = datetime.fromisoformat('2025-12-15').date()
query_start = BRUSSELS_TZ.localize(datetime.combine(date_obj - timedelta(days=1), datetime.min.time()))
query_end = BRUSSELS_TZ.localize(datetime.combine(date_obj + timedelta(days=2), datetime.min.time()))

print(f"\nQuerying reservations from {query_start.isoformat()} to {query_end.isoformat()}")

payload = {
    "Client": "Intense Experience Booking",
    "StartUtc": query_start.isoformat(),
    "EndUtc": query_end.isoformat(),
    "ServiceIds": [NIGHT_SERVICE_ID]
}

reservations_result = make_mews_request("reservations/getAll", payload)

if reservations_result and "Reservations" in reservations_result:
    reservations = reservations_result["Reservations"]
    print(f"\nFound {len(reservations)} total reservations in this time range")
    
    # Filter to only Gaia suite
    gaia_reservations = [r for r in reservations if r.get('RequestedCategoryId') == GAIA_SUITE_ID]
    print(f"Found {len(gaia_reservations)} reservations for Gaia Suite")
    
    print("\n" + "-" * 80)
    print("ALL GAIA SUITE RESERVATIONS:")
    print("-" * 80)
    
    for i, res in enumerate(gaia_reservations, 1):
        res_id = res.get('Id', 'unknown')[:8]
        res_start_utc = datetime.fromisoformat(res.get('StartUtc', '').replace('Z', '+00:00'))
        res_end_utc = datetime.fromisoformat(res.get('EndUtc', '').replace('Z', '+00:00'))
        res_start_local = res_start_utc.astimezone(BRUSSELS_TZ)
        res_end_local = res_end_utc.astimezone(BRUSSELS_TZ)
        
        print(f"\n{i}. Reservation {res_id}")
        print(f"   Start: {res_start_local.strftime('%Y-%m-%d %H:%M %Z')}")
        print(f"   End:   {res_end_local.strftime('%Y-%m-%d %H:%M %Z')}")
        print(f"   Duration: {(res_end_utc - res_start_utc).total_seconds() / 3600:.1f} hours")
    
    # Now check why December 15th night is unavailable
    print("\n" + "=" * 80)
    print("WHY IS DECEMBER 15TH NIGHT UNAVAILABLE?")
    print("=" * 80)
    
    # Define the night slot for December 15th
    dec15_night_start = BRUSSELS_TZ.localize(datetime.combine(date_obj, datetime.strptime('19:00', '%H:%M').time()))
    dec15_night_end = BRUSSELS_TZ.localize(datetime.combine(date_obj + timedelta(days=1), datetime.strptime('10:00', '%H:%M').time()))
    
    print(f"\nNight slot for December 15th:")
    print(f"  Check-in:  {dec15_night_start.strftime('%Y-%m-%d %H:%M %Z')}")
    print(f"  Check-out: {dec15_night_end.strftime('%Y-%m-%d %H:%M %Z')}")
    print(f"  With {CLEANING_BUFFER_HOURS}h buffer, needs to be free until: {(dec15_night_end + timedelta(hours=CLEANING_BUFFER_HOURS)).strftime('%Y-%m-%d %H:%M %Z')}")
    
    print(f"\nChecking for conflicts:")
    
    slot_end_with_buffer = dec15_night_end + timedelta(hours=CLEANING_BUFFER_HOURS)
    conflicts = []
    
    for res in gaia_reservations:
        res_start_utc = datetime.fromisoformat(res.get('StartUtc', '').replace('Z', '+00:00'))
        res_end_utc = datetime.fromisoformat(res.get('EndUtc', '').replace('Z', '+00:00'))
        res_start_local = res_start_utc.astimezone(BRUSSELS_TZ)
        res_end_local = res_end_utc.astimezone(BRUSSELS_TZ)
        
        # Check if reservation conflicts with the slot
        # Conflict if: reservation doesn't end before slot_end_with_buffer AND doesn't start after slot_start
        if not (res_end_local <= dec15_night_start or res_start_local >= slot_end_with_buffer):
            conflicts.append({
                'res': res,
                'start': res_start_local,
                'end': res_end_local
            })
            
            res_id = res.get('Id', 'unknown')[:8]
            print(f"\n  ❌ CONFLICT with reservation {res_id}")
            print(f"     Reservation: {res_start_local.strftime('%Y-%m-%d %H:%M')} → {res_end_local.strftime('%Y-%m-%d %H:%M')}")
            print(f"     Overlaps because:")
            if res_start_local < dec15_night_start:
                print(f"       - Reservation starts BEFORE night check-in time ({dec15_night_start.strftime('%H:%M')})")
            if res_end_local > slot_end_with_buffer:
                print(f"       - Reservation ends AFTER night check-out + buffer ({slot_end_with_buffer.strftime('%Y-%m-%d %H:%M')})")
            if res_start_local >= dec15_night_start and res_end_local <= slot_end_with_buffer:
                print(f"       - Reservation is completely within the night slot period")
            if res_start_local < slot_end_with_buffer and res_start_local >= dec15_night_start:
                print(f"       - Reservation starts during the night slot period")
    
    if not conflicts:
        print(f"\n  ✅ No conflicts found - should be available!")
    else:
        print(f"\n  Total conflicts: {len(conflicts)}")
        print(f"  Result: Night slot is BLOCKED")
else:
    print("\nFailed to fetch reservations for detailed analysis")

BULK NIGHT AVAILABILITY CHECK FOR GAIA SUITE
Suite ID: 3872869e-6278-4c64-aea8-b3850108c66f
Service: NUITEE (7ba0b732-93cc-477a-861d-b3850108b730)
Dates to check: 2
Date range: 2025-12-15 to 2025-12-16


RESULTS

Status: success

Total dates processed: 2

--------------------------------------------------------------------------------
DETAILED AVAILABILITY BY DATE:
--------------------------------------------------------------------------------

📅 Date: 2025-12-15
   Overall Available: False
   Morning Available (00:00-10:00): True
   Night Available (19:00-10:00+1): False
   Total Suites: 1
   Booked Suites: 1
   Available Suites: 0
   Booked Suite IDs: ['3872869e-6278-4c64-aea8-b3850108c66f']

📅 Date: 2025-12-16
   Overall Available: False
   Morning Available (00:00-10:00): False
   Night Available (19:00-10:00+1): False
   Total Suites: 1
   Booked Suites: 1
   Available Suites: 0
   Booked Suite IDs: ['3872869e-6278-4c64-aea8-b3850108c66f']

TIME SLOTS EXPLANATION:
Morning Slot: 0

In [15]:
import requests
from datetime import datetime, timedelta
import pytz
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Mews API configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken')
ACCESS_TOKEN = os.getenv('AccessToken')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service IDs
DAY_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"  # JOURNEE
NIGHT_SERVICE_ID = "7ba0b732-93cc-477a-861d-b3850108b730"  # NUITEE

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Get all resource categories for day service only
payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [DAY_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

print("Making API call to fetch all resource categories...")
result = make_mews_request("resourceCategories/getAll", payload)

if result and "ResourceCategories" in result:
    resource_categories = result["ResourceCategories"]
    
    # Filter for active rooms only (not suites)
    active_rooms = [
        cat for cat in resource_categories
        if cat.get("Type") == "Room" and cat.get("IsActive")
    ]
    
    print(f"\nFound {len(active_rooms)} active rooms:\n")
    
    for room in active_rooms:
        # Extract the French name from Names field
        names = room.get("Names", {})
        name = names.get("fr-FR", "Unknown Name")
        
        print(f"Name: {name}")
        print(f"ID: {room.get('Id')}")
        print(f"Service ID: {room.get('ServiceId')}")
        print(f"Classification: {room.get('Classification', 'N/A')}")
        print("-" * 50)
        
else:
    print("Failed to fetch resource categories from API")
    if result:
        print(f"Response: {result}")

Making API call to fetch all resource categories...

Found 4 active rooms:

Name: Chambre EUPHORYA
ID: f5539e51-9db0-4082-b87e-b3850108c66f
Service ID: 86fcc6a7-75ce-457a-a425-b3850108b6bf
Classification: SuperiorDouble
--------------------------------------------------
Name: Chambre IGNIS
ID: 78a614b1-199d-4608-ab89-b3850108c66f
Service ID: 86fcc6a7-75ce-457a-a425-b3850108b6bf
Classification: SuperiorDouble
--------------------------------------------------
Name: Chambre KAIROS
ID: 67ece5a2-65e2-43c5-9079-b3850108c66f
Service ID: 86fcc6a7-75ce-457a-a425-b3850108b6bf
Classification: SuperiorDouble
--------------------------------------------------
Name: Chambre AETHER
ID: 1113bcbe-ad5f-49c7-8dc0-b3850108c66f
Service ID: 86fcc6a7-75ce-457a-a425-b3850108b6bf
Classification: SuperiorDouble
--------------------------------------------------


In [9]:
import logging
import os
from datetime import datetime, timedelta
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure logging to see detailed output
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# === IMPORT CONFIGURATION AND FUNCTIONS ===
# Import all configuration from shared config file
from config import (
    ENTERPRISE_ID,
    DAY_SERVICE_ID,
    NIGHT_SERVICE_ID,
    CLEANING_BUFFER_HOURS,
    DAY_MIN_HOURS,
    DAY_MAX_HOURS,
    SPECIAL_MIN_DURATION_SUITES,
    SPECIAL_MIN_HOURS,
    ARRIVAL_TIMES,
    DEPARTURE_TIMES,
    NIGHT_CHECK_IN_HOUR,
    NIGHT_CHECK_OUT_HOUR, 
    SUITE_ID_MAPPING,
    SUITE_ID_MAPPING_REVERSE    
)

# === REAL MEWS API FUNCTION (copied from intense_experience.py) ===
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken')
ACCESS_TOKEN = os.getenv('AccessToken')

def make_mews_request(endpoint, payload):
    """Make a REAL request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })

    try:
        logger.info(f"Making REAL API request to: {url}")
        logger.info(f"Request payload: {payload}")
        
        response = requests.post(url, json=payload)
        
        logger.debug(f"Response status code: {response.status_code}")
        logger.debug(f"Response headers: {dict(response.headers)}")
        
        # Try to get response body even on error
        try:
            response_json = response.json()
            logger.debug(f"Response body: {response_json}")
        except:
            logger.debug(f"Response text: {response.text}")
            response_json = None
        
        response.raise_for_status()
        return response_json
    except requests.exceptions.HTTPError as e:
        logger.error(f"Mews API HTTP error: {e}")
        logger.error(f"Status code: {response.status_code}")
        logger.error(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        logger.error(f"Mews API request error: {e}")
        return None

# === DEMONSTRATION FUNCTION (adapted from check_bulk_availability_journee) ===
def check_bulk_availability_journee_real(make_mews_request_func, data):
    """Check availability for day bookings (journée) - REAL API VERSION with detailed logging"""
    service_id = data.get('service_id')
    dates = data.get('dates')  # List of ISO date strings
    selected_suite_id = data.get('suite_id')  # Optional: used to determine minimum duration (not for filtering)

    logger.info("=" * 80)
    logger.info("STARTING REAL BULK AVAILABILITY CHECK FOR JOURNÉE BOOKINGS")
    logger.info("=" * 80)
    logger.info(f"Service ID: {service_id}")
    logger.info(f"Dates to check: {dates}")
    logger.info(f"Selected suite ID: {selected_suite_id}")

    if not all([service_id, dates]) or len(dates) == 0:
        logger.error("Missing required parameters")
        return {"error": "Missing required parameters", "status": "error"}, 400

    # Get all suite categories for both day and night services
    logger.info("Fetching REAL suite categories from both day and night services...")
    payload = {
        "EnterpriseIds": [ENTERPRISE_ID],
        "ServiceIds": [DAY_SERVICE_ID, NIGHT_SERVICE_ID],
        "IncludeDefault": False,
        "Limitation": {"Count": 100}
    }

    suites_result = make_mews_request_func("resourceCategories/getAll", payload)
    if not suites_result or "ResourceCategories" not in suites_result:
        logger.error("Failed to fetch suites")
        return {"error": "Failed to fetch suites", "status": "error"}, 500

    all_suites = [cat for cat in suites_result["ResourceCategories"]
                  if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]
    
    logger.info(f"Found {len(all_suites)} active suites:")
    for suite in all_suites:
        # Get the name from Names field (fr-FR) or fallback to "Unknown"
        names = suite.get('Names', {})
        suite_name = names.get('fr-FR', 'Unknown')
        logger.info(f"  - {suite_name} (ID: {suite['Id']})")

    # Group suites by name to identify duplicates (e.g., GAIA suites across services)
    suite_name_groups = {}
    for suite in all_suites:
        names = suite.get('Names', {})
        suite_name = names.get('fr-FR', 'Unknown')
        if suite_name not in suite_name_groups:
            suite_name_groups[suite_name] = []
        suite_name_groups[suite_name].append(suite['Id'])
    
    logger.info("Suite name groups (for AND logic):")
    for name, ids in suite_name_groups.items():
        if len(ids) > 1:
            logger.info(f"  {name}: {ids} (will be checked together as AND)")
        else:
            logger.info(f"  {name}: {ids}")

    suite_ids = [suite["Id"] for suite in all_suites]

    # Create suite groups for AND logic - suites that must be available together
    logger.info("Creating suite groups for AND logic (mapped suites AND same-name suites must be available together)...")
    suite_groups = {}
    for suite_id in suite_ids:
        # Start with the suite itself
        suite_ids_to_check = [suite_id]
        
        # For day bookings, also check the corresponding night suite (AND rule)
        night_suite_id = SUITE_ID_MAPPING.get(suite_id)
        if night_suite_id:
            suite_ids_to_check.append(night_suite_id)
            logger.info(f"  Suite {suite_id} is mapped - adding night suite {night_suite_id}")
        
        # Also add any suites with the same name (e.g., both GAIA suites)
        for suite in all_suites:
            if suite['Id'] == suite_id:
                names = suite.get('Names', {})
                suite_name = names.get('fr-FR', 'Unknown')
                # Add all suites with the same name
                for other_suite_id in suite_name_groups.get(suite_name, []):
                    if other_suite_id not in suite_ids_to_check:
                        suite_ids_to_check.append(other_suite_id)
                        logger.info(f"  Suite {suite_id} shares name '{suite_name}' - adding {other_suite_id}")
                break
        
        # Remove duplicates
        suite_ids_to_check = list(set(suite_ids_to_check))
        suite_groups[suite_id] = suite_ids_to_check
        logger.info(f"  Final group for {suite_id}: {suite_ids_to_check}")

    logger.info(f"Found {len(suite_ids)} active suites (selected_suite_id: {selected_suite_id})")

    # Sort dates
    sorted_dates = sorted(set(dates))
    availability_results = {}
    logger.info(f"Processing dates: {sorted_dates}")

    # Process dates in chunks
    CHUNK_SIZE_DAYS = 4
    MAX_HOURS_PER_CHUNK = 96
    MAX_CONCURRENT_REQUESTS = 15

    def process_chunk(chunk_index, chunk_dates):
        """Process a single chunk of dates for day bookings - REAL API VERSION"""
        logger.info(f"Processing chunk {chunk_index + 1} with dates: {chunk_dates}")
        
        chunk_start = datetime.fromisoformat(chunk_dates[0].replace('Z', '+00:00'))
        chunk_end = datetime.fromisoformat(chunk_dates[-1].replace('Z', '+00:00'))
        chunk_end = chunk_end + timedelta(days=1)

        chunk_hours = (chunk_end - chunk_start).total_seconds() / 3600
        if chunk_hours > MAX_HOURS_PER_CHUNK:
            chunk_end = chunk_start + timedelta(hours=MAX_HOURS_PER_CHUNK)

        # For day bookings: add buffer after
        buffered_start = chunk_start.isoformat()
        buffered_end = (chunk_end + timedelta(hours=CLEANING_BUFFER_HOURS)).isoformat()

        logger.info(f"Chunk time range: {buffered_start} to {buffered_end}")

        payload = {
            "Client": "Intense Experience Booking",
            "StartUtc": buffered_start,
            "EndUtc": buffered_end,
            "ServiceIds": [DAY_SERVICE_ID, NIGHT_SERVICE_ID]
        }

        result = make_mews_request_func("reservations/getAll", payload)
        if result is None:
            logger.error(f"Failed to get reservations for chunk starting {chunk_start.date()}")
            return {}

        chunk_availability = {}

        # Get reservations (empty list if no reservations found)
        reservations = result.get("Reservations", [])
        logger.info(f"Found {len(reservations)} REAL reservations in chunk:")
        for res in reservations:
            logger.info(f"  - Suite {res['RequestedCategoryId']}: {res['StartUtc']} to {res['EndUtc']}")

        # Check each date (process all dates, even if no reservations)
        for date_str in chunk_dates:
            logger.info(f"Checking availability for date: {date_str}")
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00')).date()

            # For each suite, check which time slots are available
            suite_availability = {}

            for suite_id in suite_ids:
                suite_name = "Unknown"
                for suite in all_suites:
                    if suite["Id"] == suite_id:
                        names = suite.get('Names', {})
                        suite_name = names.get('fr-FR', 'Unknown')
                        break
                logger.info(f"  Checking suite: {suite_name} ({suite_id})")
                
                # Determine minimum hours based on the suite being checked:
                # - Each suite uses its inherent minimum (2h for special Chambres, 3h for others)
                # - UNLESS no suite is selected (golden cell scenario), then force 3h minimum for ALL
                if selected_suite_id is None:
                    # No suite selected (golden cell): use 3-hour minimum for all suites
                    min_hours = DAY_MIN_HOURS
                    logger.info(f"    No suite selected (golden cell) - using {min_hours}h minimum for all suites")
                elif suite_id in SPECIAL_MIN_DURATION_SUITES:
                    # This suite is a special Chambre: use 2-hour minimum
                    min_hours = SPECIAL_MIN_HOURS
                    logger.info(f"    Special Chambre - using {min_hours}h minimum")
                else:
                    # Regular suite: use 3-hour minimum
                    min_hours = DAY_MIN_HOURS
                    logger.info(f"    Regular suite - using {min_hours}h minimum")
                
                # Get the suite IDs that must be checked together (AND rule for mapped suites and same-name suites)
                suite_ids_to_check = suite_groups[suite_id]
                logger.info(f"    Suite group to check (AND logic): {suite_ids_to_check}")
                
                # Generate all possible time slot combinations
                available_slots = []

                for arrival_time in ARRIVAL_TIMES:
                    for departure_time in DEPARTURE_TIMES:
                        # Calculate duration
                        arr_hour = int(arrival_time.split(':')[0])
                        dep_hour = int(departure_time.split(':')[0])
                        duration = dep_hour - arr_hour

                        # Check if duration meets minimum and maximum requirements
                        if duration < min_hours or duration > DAY_MAX_HOURS:
                            logger.debug(f"    Skipping {arrival_time}-{departure_time} ({duration}h) - doesn't meet duration limits")
                            continue

                        # Create datetime objects for this slot (timezone-aware to match reservations)
                        belgian_tz = pytz.timezone('Europe/Brussels')
                        slot_start = belgian_tz.localize(datetime.combine(date_obj, datetime.strptime(arrival_time, '%H:%M').time()))
                        slot_end = belgian_tz.localize(datetime.combine(date_obj, datetime.strptime(departure_time, '%H:%M').time()))

                        logger.debug(f"    Checking slot {arrival_time}-{departure_time} ({duration}h)")

                        # Check if this slot conflicts with any reservation
                        # For mapped suites and same-name suites, check ALL suite IDs in the group (AND rule)
                        is_available = True
                        for reservation in reservations:
                            if reservation.get('RequestedCategoryId') not in suite_ids_to_check:
                                continue

                            res_start_utc = datetime.fromisoformat(reservation.get('StartUtc', '').replace('Z', '+00:00'))
                            res_end_utc = datetime.fromisoformat(reservation.get('EndUtc', '').replace('Z', '+00:00'))
                            
                            # Convert to Brussels timezone for comparison
                            res_start = res_start_utc.astimezone(belgian_tz)
                            res_end = res_end_utc.astimezone(belgian_tz)
                            
                            # Apply buffer to existing reservations (1 hour before and 1 hour after)
                            res_start_buffered = res_start - timedelta(hours=CLEANING_BUFFER_HOURS)
                            res_end_buffered = res_end + timedelta(hours=CLEANING_BUFFER_HOURS)

                            logger.debug(f"      Reservation check: {reservation.get('RequestedCategoryId')} from {res_start} to {res_end} (buffered: {res_start_buffered} to {res_end_buffered})")
                            logger.debug(f"      Slot: {slot_start} to {slot_end}")

                            # Check for overlap (new slot WITHOUT buffer vs existing reservation WITH buffer)
                            if not (slot_end <= res_start_buffered or slot_start >= res_end_buffered):
                                logger.info(f"      ❌ CONFLICT: Slot {arrival_time}-{departure_time} conflicts with reservation for suite {reservation.get('RequestedCategoryId')}")
                                is_available = False
                                break

                        if is_available:
                            logger.info(f"      ✅ AVAILABLE: Slot {arrival_time}-{departure_time} ({duration}h)")
                            available_slots.append({
                                'arrival': arrival_time,
                                'departure': departure_time,
                                'duration': duration
                            })
                        else:
                            logger.info(f"      ❌ BLOCKED: Slot {arrival_time}-{departure_time} ({duration}h)")

                suite_availability[suite_id] = available_slots
                logger.info(f"    Suite {suite_name} ({suite_id}) has {len(available_slots)} available slots")

            # Date is available if at least one suite has at least one available slot
            has_available_slot = any(len(slots) > 0 for slots in suite_availability.values())
            total_available_slots = sum(len(slots) for slots in suite_availability.values())

            chunk_availability[date_str] = {
                "available": has_available_slot,
                "total_suites": len(suite_ids),
                "suite_availability": suite_availability,
                "total_available_slots": total_available_slots
            }
            
            logger.info(f"Date {date_str} summary: available={has_available_slot}, total_slots={total_available_slots}")

        return chunk_availability

    # Create chunks
    chunks = []
    for i in range(0, len(sorted_dates), CHUNK_SIZE_DAYS):
        chunk_dates = sorted_dates[i:i + CHUNK_SIZE_DAYS]
        if chunk_dates:
            chunks.append((i // CHUNK_SIZE_DAYS, chunk_dates))

    # Process chunks in parallel
    with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_REQUESTS) as executor:
        future_to_chunk = {
            executor.submit(process_chunk, chunk_index, chunk_dates): (chunk_index, chunk_dates)
            for chunk_index, chunk_dates in chunks
        }

        for future in as_completed(future_to_chunk):
            chunk_index, chunk_dates = future_to_chunk[future]
            try:
                chunk_availability = future.result()
                availability_results.update(chunk_availability)
            except Exception as exc:
                logger.error(f"Chunk {chunk_index + 1} generated an exception: {exc}")

    logger.info("=" * 80)
    logger.info("REAL BULK AVAILABILITY CHECK COMPLETED")
    logger.info("=" * 80)
    logger.info(f"Processed {len(availability_results)} dates")

    return {
        "availability": availability_results,
        "status": "success",
        "all_suites": all_suites  # Return all_suites so it's accessible to the caller
    }

# === RUN THE REAL API DEMONSTRATION ===
if __name__ == "__main__":
    print("Fetching REAL journée availability data for December 8th, 2025")
    print("=" * 80)
    
    # Check if API tokens are available
    if not CLIENT_TOKEN or not ACCESS_TOKEN:
        print("ERROR: Please set ClientToken and AccessToken in your .env file")
        exit(1)
    
    # Test data for December 8th, 2025
    test_data = {
        'service_id': DAY_SERVICE_ID,  # JOURNEE service
        'dates': ['2025-12-08T00:00:00Z'],  # December 8th, 2025
        'suite_id': None  # No specific suite selected (golden cell scenario)
    }
    
    # Run the REAL availability check
    result = check_bulk_availability_journee_real(make_mews_request, test_data)
    
    print("\n" + "=" * 80)
    print("FINAL REAL RESULTS")
    print("=" * 80)
    
    if result.get('status') == 'success':
        availability = result.get('availability', {})
        all_suites = result.get('all_suites', [])  # Get all_suites from the result
        
        for date, data in availability.items():
            print(f"\nDate: {date}")
            print(f"  Available: {data['available']}")
            print(f"  Total suites: {data['total_suites']}")
            print(f"  Total available slots: {data['total_available_slots']}")
            
            print("  Suite details:")
            for suite_id, slots in data['suite_availability'].items():
                # Find suite name from all_suites
                suite_name = "Unknown"
                for suite in all_suites:
                    if suite["Id"] == suite_id:
                        names = suite.get('Names', {})
                        suite_name = names.get('fr-FR', 'Unknown')
                        break
                print(f"    {suite_name} ({suite_id}): {len(slots)} slots")
                for slot in slots:
                    print(f"      - {slot['arrival']} to {slot['departure']} ({slot['duration']}h)")
    else:
        print(f"Error: {result.get('error')}")

2025-11-30 11:03:51,159 - __main__ - INFO - ================================================================================
2025-11-30 11:03:51,161 - __main__ - INFO - STARTING REAL BULK AVAILABILITY CHECK FOR JOURNÉE BOOKINGS
2025-11-30 11:03:51,163 - __main__ - INFO - ================================================================================
2025-11-30 11:03:51,166 - __main__ - INFO - Service ID: 86fcc6a7-75ce-457a-a425-b3850108b6bf
2025-11-30 11:03:51,168 - __main__ - INFO - Dates to check: ['2025-12-08T00:00:00Z']
2025-11-30 11:03:51,170 - __main__ - INFO - Selected suite ID: None
2025-11-30 11:03:51,171 - __main__ - INFO - Fetching REAL suite categories from both day and night services...
2025-11-30 11:03:51,173 - __main__ - INFO - Making REAL API request to: https://api.mews-demo.com/api/connector/v1/resourceCategories/getAll
2025-11-30 11:03:51,176 - __main__ - INFO - Request payload: {'EnterpriseIds': ['c390a691-e9a0-4aa0-860c-b3850108ab4c'], 'ServiceIds': ['86fcc6a7-75c

Fetching REAL journée availability data for December 8th, 2025


2025-11-30 11:03:52,665 - urllib3.connectionpool - DEBUG - https://api.mews-demo.com:443 "POST /api/connector/v1/resourceCategories/getAll HTTP/1.1" 200 None
2025-11-30 11:03:52,670 - __main__ - DEBUG - Response status code: 200
2025-11-30 11:03:52,672 - __main__ - DEBUG - Response headers: {'Date': 'Sun, 30 Nov 2025 10:03:50 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Encoding': 'br', 'cf-cache-status': 'DYNAMIC', 'Cache-Control': 'private', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Vary': 'Accept-Encoding', 'content-security-policy': "frame-ancestors 'self' https://sales.mews-demo.com/ https://mews.seismic.com/ https://livesend.mews-demo.com/", 'request-id': '2e92e2fa-79da-4e00-88be-16dbd4643195', 'x-content-type-options': 'nosniff', 'x-frame-options': 'sameorigin', 'zp-rid': '2e92e2fa-79da-4e00-88be-16dbd4643195', 'Set-Cookie': '__cf_bm=6LM1l_GSEBWCXbMYUm8AI5aP0cU.ym


FINAL REAL RESULTS

Date: 2025-12-08T00:00:00Z
  Available: True
  Total suites: 10
  Total available slots: 6
  Suite details:
    Suite GAIA (f723bd5a-04fe-479c-bab4-b3850108c66f): 0 slots
    Chambre EUPHORYA (f5539e51-9db0-4082-b87e-b3850108c66f): 1 slots
      - 15:00 to 18:00 (3h)
    Suite EXTASE (68b87fe5-7b78-4067-b5e7-b3850108c66f): 0 slots
    Suite GAIA (3872869e-6278-4c64-aea8-b3850108c66f): 0 slots
    Chambre IGNIS (78a614b1-199d-4608-ab89-b3850108c66f): 1 slots
      - 15:00 to 18:00 (3h)
    Suite INTENSE (e4706d3a-2a06-4cb7-a449-b3850108c66f): 1 slots
      - 15:00 to 18:00 (3h)
    Suite INTENSE (f867b5c6-f62d-451c-96ec-b3850108c66f): 1 slots
      - 15:00 to 18:00 (3h)
    Chambre KAIROS (67ece5a2-65e2-43c5-9079-b3850108c66f): 1 slots
      - 15:00 to 18:00 (3h)
    Chambre AETHER (1113bcbe-ad5f-49c7-8dc0-b3850108c66f): 1 slots
      - 15:00 to 18:00 (3h)
    Suite EXTASE (0d535116-b1db-476e-8bff-b3850108c66f): 0 slots
